In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [13]:
# Load the CSV file
data_path = r'C:\stse\sems\sem8\SDP\balanced_and_standardized_images\balanced and standardized images meta for custom.csv'  # Change this to your CSV file path
image_folder = r'C:\stse\sems\sem8\SDP\balanced_and_standardized_images'  # Change this to your image folder path
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column

In [14]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)

In [15]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 26284 validated image filenames belonging to 7 classes.
Found 6570 validated image filenames belonging to 7 classes.
Found 14081 validated image filenames belonging to 7 classes.


In [16]:
# Build DenseNet121 Model
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base model layers

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(DROPOUT_RATE)(x)
output = Dense(len(label_encoder.classes_), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss="categorical_crossentropy", metrics=["accuracy"])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [17]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

C:\Users\Sudharshan Pulipaka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 408s 485ms/step - accuracy: 0.1850 - loss: 3.3778 - val_accuracy: 0.3279 - val_loss: 1.8164
Epoch 2/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 357s 434ms/step - accuracy: 0.2633 - loss: 1.8761 - val_accuracy: 0.3715 - val_loss: 1.7088
Epoch 3/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 333s 405ms/step - accuracy: 0.3047 - loss: 1.7513 - val_accuracy: 0.4011 - val_loss: 1.6138
Epoch 4/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 350s 426ms/step - accuracy: 0.3237 - loss: 1.6872 - val_accuracy: 0.4233 - val_loss: 1.5395
Epoch 5/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 351s 427ms/step - accuracy: 0.3494 - loss: 1.6333 - val_accuracy: 0.4306 - val_loss: 1.4994
Epoch 6/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 467s 568ms/step - accuracy: 0.3519 - loss: 1.6135 - val_accuracy: 0.4472 - val_loss: 1.4638
Epoch 7/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 355s 431ms/step - accuracy: 0.3755 - loss: 1.5743 - val_accuracy: 0.4633 - val_loss: 1.4271
Epoch 8/20
822/822 ━━━━━━━━━━━━━━━━━━━━ 357s 434ms/step - accuracy: 0.3833 -

In [18]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [19]:
plot_model_training_curve(history)

In [20]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

C:\Users\Sudharshan Pulipaka\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



441/441 ━━━━━━━━━━━━━━━━━━━━ 108s 244ms/step - accuracy: 0.5156 - loss: 1.2673

Test Accuracy: 51.67%


In [ ]:
model.save(r'C:\stse\sems\sem8\SDP\pretrained\densenet\densenet 70-30\densenet 70-30.keras') 

In [ ]:
model.save_weights(r'C:\stse\sems\sem8\SDP\pretrained\densenet\densenet 70-30\densenet 70-30.weights.h5') 